In [ ]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [ ]:
sizes = [10, 20, 50, 100, 200, 500, 1_000]
densities = [.2, .5, .8, 1.0]
noise_levels = [.0, .1, .5, .8]
negative_values_options = [False, True]

In [ ]:
problem = create_problem(20, density=0.2, noise_level=0.1, negative_values=False)
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
def solve_approximate(G: nx.DiGraph, landmark_fraction: float, only_positive :bool = False) -> np.ndarray:
    reversed_G = G.reverse(copy=True)
    N = G.number_of_nodes()
    N_landmarks = int(landmark_fraction * N)
    landmarks = np.random.choice(N, size=N_landmarks, replace=False).tolist()

    approx_distances = np.zeros((N, N)) + np.inf
    from_landmark = np.zeros((N_landmarks, N)) + np.inf
    to_landmark = np.zeros((N_landmarks, N)) + np.inf

    for i, lm in enumerate(landmarks):
        try:
            length, path = nx.single_source_bellman_ford(G, lm, weight='weight')
            for node, dist in length.items():
                approx_distances[lm, node] = dist
                from_landmark[i, node] = dist
            length, path = nx.single_source_bellman_ford(reversed_G, lm, weight='weight')
            for node, dist in length.items():
                to_landmark[i, node] = dist
        except nx.NetworkXUnbounded:
            # Negative cycle detected
            continue
    
    if only_positive:
        to_landmark[to_landmark < 0] = np.inf
        from_landmark[from_landmark < 0] = np.inf
        approx_distances[approx_distances < 0] = np.inf

    for source in range(N):
        for dest in range(N):
            if source == dest or not np.isinf(approx_distances[source, dest]):
                continue
            distances = to_landmark[:, source] + from_landmark[:, dest]
            approx_distances[source, dest] = np.min(distances)
    np.fill_diagonal(approx_distances, np.inf)
    return approx_distances

In [ ]:
def solve_exact(G: nx.DiGraph, only_positive: bool = False) -> np.ndarray:
    N = G.number_of_nodes()
    all_distances = np.zeros((N, N)) + np.inf
    for s, d in product(range(N), repeat=2):
        if s == d:
            continue
        try:
            # path = nx.shortest_path(G, s, d, weight='weight')
            path = nx.bellman_ford_path(G, s, d, weight='weight')
            cost = cost = nx.path_weight(G, path, weight='weight')
        except nx.NetworkXNoPath:
            # Nodes are not connected
            path = None
            cost = np.inf
        except nx.NetworkXUnbounded:
            # Negative cycle detected
            path = None
            cost = -np.inf
        if only_positive and cost < 0:
            cost = np.inf
        all_distances[s, d] = cost
    return all_distances

In [ ]:
iterator = product(sizes, densities, noise_levels, negative_values_options)
iterator_size = len(sizes)*len(densities)*len(noise_levels)*len(negative_values_options)
results = {}
for size, density, noise_level, negative_values in tqdm(iterator, total=iterator_size):
    ic(size, density, noise_level, negative_values)
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=42,
    )
    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    approximate_distances = solve_approximate(G, landmark_fraction=0.1, only_positive=negative_values)
    results[(size, density, noise_level, negative_values)] = approximate_distances